### Remove Users from Power User profiles ###
The purpose of this script is to remove the assignment of users to Power Users. When a user is added to Docebo by a Power User, that new user is automatically assigned to the Power User's profile. This is problematic because even if a user is moved between branches, the Power User who added them still has control over their profile. There is no way to remove the automatic assignment, so users must be periodically purged from the Power User assignments (ideally weekly).

After updating the Client information, this script can be run as is to purge users from Power Users.

In [ ]:
import pandas as pd
import requests
import json
import math
import os
from dotenv import load_dotenv

In [ ]:
#Client information
load_dotenv()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [ ]:
#Get authorization token
#Data payload for token request
token_data = {'client_id':client_id,'client_secret':client_secret,'grant_type':'password', 'username':username, 'password':password, 'scope':'api'}

p1 = requests.post('https://scooterscoffee.docebosaas.com/oauth2/token', data=token_data)

p1_result = json.loads(p1.text)

print(p1_result)

In [ ]:
#Access token information
access_token = p1_result['access_token']

In [ ]:
#Headers 
headers = {'Accept':'application/json','Client-ID':client_id, 'Authorization':'Bearer ' + access_token}

In [ ]:
#Get list of Power Users

#API endpoint for getting Power Users
pu_get_URL = 'https://scooterscoffee.docebosaas.com/poweruser/v1/powerusers'
pageSize = 200

requestPayload1 = requests.get(pu_get_URL + '?page_size=' + str(pageSize), headers=headers)
requestJSON1 = requestPayload1.json()

#Get total number of users
recordCount = int(requestJSON1['data']['total_count'])

print("Total Power Users: " + str(requestJSON1['data']['total_count']))

#Calculate the number of loops needed to retrieve all records
loopCounter = recordCount / 200

#Round up loopCounter
looper = math.ceil(loopCounter)

In [ ]:
#Query all possible records for Power Users

#Initialize dataframe
powerUser_df = pd.DataFrame()

#Initialize page number
pageNum = 1

#Get all pages of data
for i in range(looper):
    requestPayload2 = requests.get(pu_get_URL + '?page_size=' + str(pageSize) + '&page=' + str(pageNum), headers=headers)
    requestJSON2 = requestPayload2.json()
    print("Page " + str(pageNum) + " has finished its call.")
    pageNum += 1
    itemsDict = requestJSON2['data']['items']
    itemsDict_df = pd.DataFrame.from_dict(itemsDict)
    powerUser_df = pd.concat([powerUser_df, itemsDict_df], ignore_index=True)
    

In [ ]:
#Build simplified dataframe to get Power Users with assigned users

#Get simplified list of all Power Users
powerUser_simple_df = powerUser_df[['id','username','firstname','lastname','total_assigned_users']]

#Create dataframe of users with assigned users. (This is to help cut down on API transactions for Power Users without assigned users.)
pu_withAssignedUsers_df = powerUser_simple_df[powerUser_simple_df['total_assigned_users'] > 0]


#Create list of Powers Users with assigned users by ID Number
pu_id_withAssignedUsers_list = list(pu_withAssignedUsers_df['id'])


#Show number of records to clean up
print(str(len(pu_id_withAssignedUsers_list)) + " users to update.")

In [ ]:
#Run loop to remove assigned users from all Power Users

#Delete statement for API call. Users string formatting with list of Power Users with assigned users. The {0} in the url is the substituted value.
pu_delete_url = 'https://scooterscoffee.docebosaas.com/poweruser/v1/powerusers/{0}/users'

#Additional payload information. Must pass "All:true" to remove all users on the delete request or it won't remove any. This must be in a json.dumps() to accomodate Docebo.
payload = json.dumps({"all":'true'})

#Create variables for list counting in print
curRecord = 1

#Loop through list of user IDs and substitute them into the DELETE request each time.
for user in pu_id_withAssignedUsers_list:
    tempDeleteURL = pu_delete_url.format(user)
    requestPayload3 = requests.delete(tempDeleteURL, data=payload, headers=headers)
    requestJSON3 = requestPayload3.json()
    print("API request: " + tempDeleteURL)
    print(requestPayload3)
    print("Finished record "+ str(curRecord) + " of " + str(len(pu_id_withAssignedUsers_list)))
    curRecord += 1

print("Update complete.")